In [7]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import yaml
from tqdm import tqdm

In [8]:
login_payload = {
    'id__email': 'dmcdade@insightsquared.com',
    'id__password': 'skLETTY2011#'
}

In [9]:
def get_token(session, login_url='https://clients.insightsquared.com/login/'):
    # get the unique token code
    login_resp = session.get(login_url)
    return BeautifulSoup(login_resp.text, 'lxml').find('input', attrs={'name': 'csrfmiddlewaretoken'}).attrs['value']

def login(payload=login_payload):
    # begin a web session and visit the IS2 webpage
    session = requests.session()
    login_url='https://clients.insightsquared.com/login/'
    token = get_token(session)
    payload['csrfmiddlewaretoken'] = token  # append token to payload dict
    # get an authenticated session and verity results
    login_response = session.post(
        login_url, 
        data=payload, 
        headers=dict(referer=login_url)
    )
    # check that the login worked
    print BeautifulSoup(login_response.text, 'lxml').find('title')
    print get_client_details(login_response)
    return session, token

def get_client_details(base_page):
    client_picker = BeautifulSoup(base_page.text, 'lxml').find('client-picker').attrs['params']
    client_yaml = yaml.load(client_picker[16:].replace('}', '') + '}')  # need to deal with crummy string formatting
    # client_yaml['abbreviation'], client_yaml['id'], client_yaml['name'], client_yaml['site'] 
    return client_yaml

In [10]:
session, token = login()

<title> My Dashboard | InsightSquared</title>
{'abbreviation': 'CDI', 'id': 3561, 'name': 'CDI', 'site_url': 'https://clients.insightsquared.com/'}


In [11]:
def get_available_tags(auth_session=session):
    url = 'https://clients.insightsquared.com/settings/tags/Employee'
    response = auth_session.get(url)
    page_soup = BeautifulSoup(response.text, 'lxml')
    existing_tags = page_soup.find('ul', attrs={'class': 'available_tags'}).findAll('p', attrs={'class': 'tag_name'})
    return [tag.text for tag in existing_tags]

def get_user_selected_tags(auth_session=session):
    url = 'https://clients.insightsquared.com/settings/tags/Employee'
    response = auth_session.get(url)
    page_soup = BeautifulSoup(response.text, 'lxml')
    employee_tag_list = []
    # get each of the employees and the details
    employee_tag_table = page_soup.find('table', attrs={'class': 'blist clear'}).findAll('tr')[1:]
    for employee_row in employee_tag_table:
        employee_details_list = []
        employee, emp_id = [col.text for col in employee_row.findAll('td', attrs={'class': 'name_column'})]
        tag_list = employee_row.findAll('li')
        for tag in tag_list:
            selected = True if tag.attrs['class'] == ['selected_tag', 'tag'] else False
            url = tag.find('a').attrs['data-href']
            tag_name = tag.find('a').text
            employee_details_list = employee_details_list + [[employee, emp_id, tag_name, selected, url]]
        employee_tag_list = employee_tag_list + employee_details_list
    return pd.DataFrame(employee_tag_list, columns=['name', 'id', 'tag_name', 'selected', 'url'])

In [12]:
# https://clients.insightsquared.com/settings/users/access_groups/
tag_page_url = 'https://clients.insightsquared.com/settings/tags/Employee'
tags = session.get(tag_page_url)
client_details_yaml = get_client_details(tags)  # get the client id and client name for the portal that we are currently in

# check some of the values 

In [259]:
existing_tags = get_available_tags()

[u'AECOM URS',
 u'Applications',
 u'Applications and Project Resources',
 u'Apps Development',
 u'Arizona',
 u'ATL Recruiting Multiple',
 u'Atlanta',
 u'Beth Alex',
 u'Bombardier Ericsson',
 u'Business Inteligence',
 u'Business Intelligence',
 u'Calgary',
 u'California',
 u'Canada',
 u'Canada Staffing',
 u'Cary NC',
 u'Cindy Greenbaum',
 u'Cross Lanes',
 u'Dallas',
 u'Darren Simmons',
 u'Direct Hire',
 u'Direct Hire Recruiter',
 u'Eastern Canada',
 u'Edgerock',
 u'Edgerock Lite',
 u'Edgerock Lite Class 2',
 u'EdgeRock National Sales',
 u'Emerson Curtis Wright',
 u'Emerson Lyondell AECOM',
 u'Engagement Lead',
 u'Engineering',
 u'Enterprise',
 u'Enterprise IBM',
 u'Enterprise Non IBM',
 u'Enterprise Program',
 u'Ericsson',
 u'Ericsson IBM',
 u'Erin Medlin',
 u'ERP',
 u'ERP Business Intel',
 u'GETS',
 u'Grand Rapids',
 u'Gulf Atlantic Multiple',
 u'HCL',
 u'Hugo Milan',
 u'IBM',
 u'IBM Siemens',
 u'Illinois',
 u'Indiana',
 u'Infrastructure',
 u'Irvine',
 u'Jr Recruiter',
 u'Junior Recrui

In [260]:
employee_tag_list = get_user_selected_tags()

In [261]:
employee_tag_list.head()

,name,id,tag_name,selected,url
0,Aaron Yuhas,9145459,AECOM URS,False,/tag/toggle/AECOM%20URS/Employee/BH_9145459/
1,Aaron Yuhas,9145459,Applications,False,/tag/toggle/Applications/Employee/BH_9145459/
2,Aaron Yuhas,9145459,Applications and Project Resources,False,/tag/toggle/Applications%20and%20Project%20Res...
3,Aaron Yuhas,9145459,Apps Development,False,/tag/toggle/Apps%20Development/Employee/BH_914...
4,Aaron Yuhas,9145459,Arizona,False,/tag/toggle/Arizona/Employee/BH_9145459/


In [262]:
employee_tag_list[employee_tag_list['selected']].head()  # which filters are selected for specific users

,name,id,tag_name,selected,url
19,Aaron Yuhas,9145459,Darren Simmons,True,/tag/toggle/Darren%20Simmons/Employee/BH_9145459/
33,Aaron Yuhas,9145459,Enterprise Non IBM,True,/tag/toggle/Enterprise%20Non%20IBM/Employee/BH...
44,Aaron Yuhas,9145459,Hugo Milan,True,/tag/toggle/Hugo%20Milan/Employee/BH_9145459/
60,Aaron Yuhas,9145459,Major Infrastructure,True,/tag/toggle/Major%20Infrastructure/Employee/BH...
65,Aaron Yuhas,9145459,Midwest Multiple,True,/tag/toggle/Midwest%20Multiple/Employee/BH_914...


# Formatting and headers 

In [13]:
session_cookies = session.cookies.get_dict()

cookies = {
    'csrftoken': session_cookies['csrftoken'],
    '__utma': '99182412.1401600144.1444310363.1464213977.1464711194.9',
    '__utmz': '99182412.1454080917.6.5.utmcsr=insightsquared.com|utmccn=(referral)|utmcmd=referral|utmcct=/saas-benchmarking-2016/',
    'is2_welcome_tour': '%7B%22is_open%22%3A%22%22%2C%22current_step%22%3A%22%22%7D',
    'optimizelySegments': '%7B%221022955017%22%3A%22ff%22%2C%221024034951%22%3A%22false%22%2C%221030194866%22%3A%22referral%22%7D',
    'optimizelyEndUserId': 'oeu1448979321867r0.48276709385018957',
    'optimizelyBuckets': '%7B%223507080702%22%3A%223521680223%22%7D',
    'insight': '%7B%22referrer%22%3A%22Direct%22%7D',
    '_mkto_trk': 'id:054-ACZ-798&token:_mch-insightsquared.com-1448979324826-81180',
    '_ga': 'GA1.2.1718304047.1448979325',
    'ei_client_id': '570f88657962fb1100fb903e',
    'mp_b7fbb2e376692a71698232012abd84f1_mixpanel': '%7B%22distinct_id%22%3A%20%221515de5d9c753f-0b5bcf759bdab38-46534131-100200-1515de5d9c8612%22%2C%22%24initial_referrer%22%3A%20%22%24direct%22%2C%22%24initial_referring_domain%22%3A%20%22%24direct%22%2C%22utm_medium%22%3A%20%22email%22%7D',
    '__cfduid': 'dfc858bb8c30f61f564efc124352ea9561458741396',
    'sessionid': session_cookies['sessionid'],
    '__utmb': '99182412.3.10.1464711194',
    '__utmc': '99182412',
    '__utmt': '1',
}

headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Host': 'clients.insightsquared.com',
    'Referer': 'https://clients.insightsquared.com/settings/tags/Employee/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:46.0) Gecko/20100101 Firefox/46.0',
    'X-CSRFToken': session_cookies['csrftoken'],
    'X-ClientId': client_details_yaml['id'],
    'X-Requested-With': 'XMLHttpRequest',
}

# Create New Tag

In [ ]:
### When the name has spaces use '+'

In [216]:
tag_data = 'tagName=temp+tag21'
add_tag_url = 'https://clients.insightsquared.com/tag/add/'

post_resp = session.post(add_tag_url, headers=headers, cookies=cookies, data=tag_data)
print post_resp

<Response [200]>


# Delete Tag 

### when the name has spaces use '%20' in space

In [236]:
tag_name = 'temp tag21'  # spaces 'new test 3' -> 'new%20test%203'
#tag_name = 'finalv3'
delete_tag_url = 'https://clients.insightsquared.com/tag/delete/{0}/'.format(tag_name.replace(' ', '%20'))

post_resp = session.post(delete_tag_url, headers=headers, cookies=cookies, data=tag_data)
print post_resp

<Response [200]>


In [277]:
existing_tags = get_available_tags()

In [278]:
existing_tags[0:10]

[u'GETS',
 u'Team Anielka Barrow',
 u'Team Brian Mauch',
 u'Team Catherine Belgrave',
 u'Team Crystal Meyers']

In [279]:
tag_page_url = 'https://clients.insightsquared.com/settings/tags/Employee'
# tags = session.get(tag_page_url)
client_details_yaml = get_client_details(tags)
client_details_yaml

{'abbreviation': 'CDI',
 'id': 3561,
 'name': 'CDI',
 'site_url': 'https://clients.insightsquared.com/'}

In [280]:
for tag_name in tqdm(existing_tags):
    if tag_name != 'GETS':
        delete_tag_url = 'https://clients.insightsquared.com/tag/delete/{0}/'.format(tag_name.replace(' ', '%20'))
        post_resp = session.post(delete_tag_url, headers=headers, cookies=cookies, data=tag_data)
        print post_resp

 40%|█████████████████████████████████████▏                                                       | 2/5 [00:18<00:28,  0.11it/s]

<Response [200]>
<Response [200]>
<Response [200]>

 80%|██████████████████████████████████████████████████████████████████████████▍                  | 4/5 [00:59<00:11,  0.09it/s]


<Response [200]>

# Select the tag for a user

In [141]:
base_tag_url = 'https://clients.insightsquared.com/'
select_tag_url = 'tag/toggle/temp%20tag/Employee/PVTR_1665306/'
url = base_tag_url + selected_tag_url
post_resp = session.post(url, headers=headers, cookies=cookies, data=tag_data)
print post_resp

<Response [200]>


In [ ]:
base_tag_url = 'https://clients.insightsquared.com/'
select_tag_url = 'tag/toggle/temp%20tag/Employee/PVTR_1665306/'
url = base_tag_url + selected_tag_url
post_resp = session.post(url, headers=headers, cookies=cookies, data=tag_data)
print post_resp

In [233]:
for url in employee_tag_list.ix[subset].url.values:
    base_tag_url = 'https://clients.insightsquared.com'
    post_resp = session.post(base_tag_url + url, headers=headers, cookies=cookies, data=tag_data)
    print post_resp

<Response [200]>
<Response [200]>
<Response [200]>


In [234]:
employee_tag_list = get_user_selected_tags()

In [235]:
employee_tag_list[employee_tag_list['selected']]

,name,id,tag_name,selected,url
168,Daniel Tyson,1706388,ProServ,True,/tag/toggle/Client%20Services/Employee/PVTR_17...
213,Emmanuel Runes,1036999,ProServ,True,/tag/toggle/Client%20Services/Employee/PVTR_10...
246,Fredus Loves Goats,005E0000001hbj2IAA,Foxtrot,True,/tag/toggle/Foxtrot/Employee/SFDC_005E0000001h...
273,Georgia Wright-Simmons,1279498,ProServ,True,/tag/toggle/Client%20Services/Employee/PVTR_12...
343,Jonathan Anderson,1408506,ProServ,True,/tag/toggle/Client%20Services/Employee/PVTR_14...
533,Mike Swartz,1740284,ProServ,True,/tag/toggle/Client%20Services/Employee/PVTR_17...


# Read the excel file contents

In [14]:
xlfile = pd.ExcelFile('TTS Practice Model - Recruiting Delivery Team Structure (Final) 5-24-16 copy to Insight_updt.xlsx')
data = pd.read_excel(xlfile, sheetname='is2_data')
data['ID'] = data['ID'].astype(str).apply(lambda x: x.split('.')[0])
data.head()

,ID,Name,Role,Major,Minor,Fulfillment Practice Manager,Practice Team,Fulfillment Practice Lead,Fulfillment Practice Coordinator,US/Canada,Work Location,Onshore/Offshore,TTS Division,Client/Region,New Hires,Targets Yes/No,GETS
0,9121540,Jeffrey Prukop,Recruiter,Tech Support,Project Resources,Amy Nelson,Infrastructure,Erin Medlin,Cindy Greenbaum,US,Virtual,On Shore,Enterprise,IBM,12 Weeks +,Yes,NaN
1,9071639,Alfred Isaac,Recruiter,Project Resources,Other/General,NaN,NaN,NaN,NaN,US,Virtual,Off Shore,Enterprise,IBM,12 Weeks +,Yes,NaN
2,9145459,Aaron Yuhas,Recruiter,Infrastructure,Apps Development,Amy Nelson,Infrastructure,Erin Medlin,Cindy Greenbaum,US,Virtual,On Shore,Enterprise,Midwest - Multiple,12 Weeks +,Yes,NaN
3,178,Chris Martin,Recruiter,Apps Development,Tech Support,NaN,NaN,NaN,NaN,US,Virtual,Off Shore,Enterprise,IBM,12 Weeks +,Yes,NaN
4,9390427,Deona Williams,Jr. Recruiter,ERP,Business Inteligence,Amy Nelson,Infrastructure,Erin Medlin,Cindy Greenbaum,US,Atlanta,On Shore,Enterprise,Edgerock Lite,New E/L- 12 Week Ramp,Yes,NaN


In [15]:
print [str(col) for col in data.columns]

['ID', 'Name', 'Role', 'Major', 'Minor', 'Fulfillment Practice Manager', 'Practice Team', 'Fulfillment Practice Lead', 'Fulfillment Practice Coordinator', 'US/Canada', 'Work Location', 'Onshore/Offshore', 'TTS Division', 'Client/Region', 'New Hires', 'Targets Yes/No', 'GETS']


In [32]:
filter_cols = ['Role', 'Major','Minor','Fulfillment Practice Manager', 
               'Practice Team','Fulfillment Practice Lead', 'Fulfillment Practice Coordinator',
               'US/Canada', 'Work Location','Onshore/Offshore',
               'TTS Division', 'Client/Region', 'New Hires',
               'Targets Yes/No', 'GETS']
new_vals = []
for col in filter_cols:
    data[col] = data[col].apply(
        lambda x: str(x).replace('/', ' ').replace(' - ', ' ').replace('-', '').replace(',', '').replace('.', '').replace('+', 'plus')
        if not pd.isnull(x) else x
    )
    data_vals = data[col].dropna().drop_duplicates().astype(str).values.tolist()
    for val in data_vals:
        new_vals = new_vals + [col.replace('/', ' or ') + ': ' + val]
new_vals

['Role: Recruiter',
 'Role: Jr Recruiter',
 'Role: Sourcer',
 'Role: Team Lead',
 'Role: Direct Hire',
 'Role: Lead Recruiter',
 'Role: Backup Recruiter',
 'Role: Engagement Lead',
 'Major: Tech Support',
 'Major: Project Resources',
 'Major: Infrastructure',
 'Major: Apps Development',
 'Major: ERP',
 'Major: Engineering',
 'Major: Other General',
 'Major: Business Inteligence',
 'Major: Business Intelligence',
 'Minor: Project Resources',
 'Minor: Other General',
 'Minor: Apps Development',
 'Minor: Tech Support',
 'Minor: Business Inteligence',
 'Minor: Infrastructure',
 'Minor: ERP',
 'Minor: Engineering',
 'Fulfillment Practice Manager: Amy Nelson',
 'Fulfillment Practice Manager: Keeran Persaud',
 'Fulfillment Practice Manager: Brian Mauch',
 'Fulfillment Practice Manager: Crystal Meyers',
 'Fulfillment Practice Manager: Jim Godfrey',
 'Fulfillment Practice Manager: Taylor Grimes',
 'Fulfillment Practice Manager: Nikki Diman',
 'Fulfillment Practice Manager: Taylor Grimes ',
 'Fu

In [33]:
new_vals[-1] = 'GETS'

In [34]:
print new_vals

['Role: Recruiter', 'Role: Jr Recruiter', 'Role: Sourcer', 'Role: Team Lead', 'Role: Direct Hire', 'Role: Lead Recruiter', 'Role: Backup Recruiter', 'Role: Engagement Lead', 'Major: Tech Support', 'Major: Project Resources', 'Major: Infrastructure', 'Major: Apps Development', 'Major: ERP', 'Major: Engineering', 'Major: Other General', 'Major: Business Inteligence', 'Major: Business Intelligence', 'Minor: Project Resources', 'Minor: Other General', 'Minor: Apps Development', 'Minor: Tech Support', 'Minor: Business Inteligence', 'Minor: Infrastructure', 'Minor: ERP', 'Minor: Engineering', 'Fulfillment Practice Manager: Amy Nelson', 'Fulfillment Practice Manager: Keeran Persaud', 'Fulfillment Practice Manager: Brian Mauch', 'Fulfillment Practice Manager: Crystal Meyers', 'Fulfillment Practice Manager: Jim Godfrey', 'Fulfillment Practice Manager: Taylor Grimes', 'Fulfillment Practice Manager: Nikki Diman', 'Fulfillment Practice Manager: Taylor Grimes ', 'Fulfillment Practice Manager: Sherr

# Create the tags

In [35]:
for filter_val in tqdm(new_vals):
    tag_data = 'tagName={0}'.format(filter_val.replace(' ', '+'))
    add_tag_url = 'https://clients.insightsquared.com/tag/add/'
    post_resp = session.post(add_tag_url, headers=headers, cookies=cookies, data=tag_data)
    print post_resp

  1%|▉                                                                                          | 1/101 [00:00<00:40,  2.49it/s]

<Response [200]>
<Response [200]>

  2%|█▊                                                                                         | 2/101 [00:00<00:43,  2.26it/s]


<Response [200]>

  3%|██▋                                                                                        | 3/101 [00:01<00:37,  2.59it/s]

...


<Response [200]>

 98%|████████████████████████████████████████████████████████████████████████████████████████▏ | 99/101 [00:26<00:00,  4.72it/s]


<Response [200]>

 99%|████████████████████████████████████████████████████████████████████████████████████████ | 100/101 [00:26<00:00,  5.05it/s]


<Response [200]>

# apply the tags to the employees

In [36]:
employee_tags = get_user_selected_tags()

In [37]:
employee_tags.head()

,name,id,tag_name,selected,url
0,Aaron Yuhas,9145459,Client or Region: AECOM URS,False,/tag/toggle/Client%20or%20Region%3A%20AECOM%20...
1,Aaron Yuhas,9145459,Client or Region: ATL Recruiting Multiple,False,/tag/toggle/Client%20or%20Region%3A%20ATL%20Re...
2,Aaron Yuhas,9145459,Client or Region: Bombardier Ericsson,False,/tag/toggle/Client%20or%20Region%3A%20Bombardi...
3,Aaron Yuhas,9145459,Client or Region: Eastern Canada,False,/tag/toggle/Client%20or%20Region%3A%20Eastern%...
4,Aaron Yuhas,9145459,Client or Region: Edgerock Lite,False,/tag/toggle/Client%20or%20Region%3A%20Edgerock...


In [38]:
employee_tags[employee_tags['selected']].head(2)

,name,id,tag_name,selected,url
14,Aaron Yuhas,9145459,Client or Region: Midwest Multiple,True,/tag/toggle/Client%20or%20Region%3A%20Midwest%...
29,Aaron Yuhas,9145459,Fulfillment Practice Coordinator: Cindy Greenbaum,True,/tag/toggle/Fulfillment%20Practice%20Coordinat...


In [39]:
for employee in tqdm(data['Name']):
    employee_data = data[data['Name'] == employee]
    individual_tags = employee_tags[
        employee_tags['name'].apply(lambda x: x.replace("'", '').replace(' ', '').lower()) == employee.replace("'", '').replace(' ', '').lower()
    ]
    for col in filter_cols:
        val = employee_data[col].values[0]
        if not pd.isnull(val):
            employee_col_filter = col.replace('/', ' or ') + ': ' + val
            employee_col_filter = 'GETS' if employee_col_filter == 'GETS: GETS' else employee_col_filter
            individual_tag_employee_col = individual_tags[individual_tags['tag_name'] == employee_col_filter]
            if not individual_tag_employee_col.empty and individual_tag_employee_col['selected'].values[0] != True:
                url = individual_tag_employee_col['url'].values[0]
                base_tag_url = 'https://clients.insightsquared.com'
                post_resp = session.post(base_tag_url + url, headers=headers, cookies=cookies, data=tag_data)
                print post_resp

  1%|▌                                                                                          | 1/177 [00:00<01:08,  2.55it/s]

<Response [200]>
<Response [200]>

  2%|█▌                                                                                         | 3/177 [00:00<00:52,  3.33it/s]


<Response [200]>

  2%|██                                                                                         | 4/177 [00:01<00:53,  3.24it/s]

...


<Response [200]>

 90%|███████████████████████████████████████████████████████████████████████████████▉         | 159/177 [00:38<00:05,  3.41it/s]


<Response [200]>

 90%|████████████████████████████████████████████████████████████████████████████████▍        | 160/177 [00:39<00:04,  3.48it/s]


<Response [200]>

In [40]:
employee_tags = get_user_selected_tags()

In [41]:
employee_tags[employee_tags['selected']].head(2)

,name,id,tag_name,selected,url
14,Aaron Yuhas,9145459,Client or Region: Midwest Multiple,True,/tag/toggle/Client%20or%20Region%3A%20Midwest%...
29,Aaron Yuhas,9145459,Fulfillment Practice Coordinator: Cindy Greenbaum,True,/tag/toggle/Fulfillment%20Practice%20Coordinat...


In [47]:
employee_tags[(employee_tags.name == 'Jean Dumlao') & (employee_tags.selected)].ix[9061].tag_name

u'New Hires: 12 Weeks  plus '

In [43]:
existing_tags = get_available_tags()
existing_tags

[u'Client or Region: AECOM URS',
 u'Client or Region: ATL Recruiting Multiple',
 u'Client or Region: Bombardier Ericsson',
 u'Client or Region: Eastern Canada',
 u'Client or Region: Edgerock Lite',
 u'Client or Region: Edgerock Lite Class 2',
 u'Client or Region: Emerson Curtis Wright',
 u'Client or Region: Emerson Lyondell AECOM',
 u'Client or Region: Ericsson',
 u'Client or Region: Ericsson IBM',
 u'Client or Region: Gulf Atlantic Multiple',
 u'Client or Region: HCL',
 u'Client or Region: IBM',
 u'Client or Region: IBM Siemens',
 u'Client or Region: Midwest Multiple',
 u'Client or Region: Multi Clients Direct Hire',
 u'Client or Region: Parker',
 u'Client or Region: Parker LI',
 u'Client or Region: Schneider Electric Invensys',
 u'Client or Region: Schneider Electric Invensys Teradyne LI',
 u'Client or Region: Siemens',
 u'Client or Region: Siemens Phillips',
 u'Client or Region: Tech Sol Multiple',
 u'Client or Region: Teradyne',
 u'Client or Region: UTC Disney Lyondell',
 u'Client 